In [1]:
import psycopg2
from config import config

In [2]:
def connect():
    '''Connect to the PostGresQL Database Server'''
    conn = None
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        print(db_version)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__=='__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.5.25 on x86_64-apple-darwin21.1.0, compiled by Apple clang version 13.0.0 (clang-1300.0.29.3), 64-bit',)
Database connection closed.


In [17]:
def get_info(query):
    '''this function queries the database and find the information specified in the parameter'''
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        return results
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [18]:
# what is the email address for the customer with the name Nancy Thomas?
get_info("SELECT email FROM customer WHERE first_name = 'Nancy' AND last_name = 'Thomas'")

[('nancy.thomas@sakilacustomer.org',)]

In [19]:
# give a customer the description of the movie "Outlaw Hanky"
get_info("SELECT description FROM film WHERE title = 'Outlaw Hanky';")

[('A Thoughtful Story of a Astronaut And a Composer who must Conquer a Dog in The Sahara Desert',)]

In [20]:
# get the customer number for the customer who lives at 259 Ipoh Drive
get_info("SELECT phone FROM address WHERE address = '259 Ipoh Drive';")

[('419009857119',)]

In [21]:
# get more information about the customer who lives at 259 Ipoh Drive
get_info("""SELECT 
            c.first_name, c.last_name, c.email, a.address, a.phone 
        FROM customer c 
            LEFT JOIN 
                    address a 
                ON c.address_id = a.address_id 
            WHERE a.address = '259 Ipoh Drive';""")

[('Jesus',
  'Mccartney',
  'jesus.mccartney@sakilacustomer.org',
  '259 Ipoh Drive',
  '419009857119')]

In [22]:
# what are the customer ids of the 10 first customers who created a payment?
get_info("SELECT customer_id from payment ORDER BY payment_date ASC LIMIT 10;")

[(416,), (516,), (239,), (592,), (49,), (264,), (46,), (481,), (139,), (595,)]

In [23]:
# what are the titles of the 5 shortest movies?
get_info("SELECT title, length FROM film ORDER BY length ASC LIMIT 5;")

[('Iron Moon', 46),
 ('Kwai Homeward', 46),
 ('Alien Center', 46),
 ('Ridgemont Submarine', 46),
 ('Labyrinth League', 46)]

In [24]:
# how many movies can a customer watch that are less than or equal to 50 minutes long?
get_info("SELECT COUNT(*) FROM film WHERE length <= 50;")

[(37,)]